In [290]:
import numpy as np
import pandas as pd
import math

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.preprocessing import normalize
from scipy.sparse import dia_matrix
from scipy.optimize import minimize, shgo, dual_annealing, differential_evolution, basinhopping
import scipy

from sklearn.model_selection import train_test_split

In [13]:
path = "../data/data.xlsx"
df = pd.read_excel(path)
df.Дата = df.Дата.astype('datetime64[ns]')

In [14]:
features = [
    ['intercept', 'Кривизна'],
    ['intercept', 'Кривизна', 'Профиль пути'],
    ['intercept', 'Кривизна', 'n_feature_1'], # Профиль пути * Макс. число вагонов в сходе
    ['intercept', 'Кривизна', 'n_feature_2'], # 1 - Макс. число вагонов в сходе / Общее количество вагонов
    ['intercept', 'Кривизна', 'n_feature_3'], # Скорость * Загрузка
    ['intercept', 'Кривизна', 'Профиль пути', 'n_feature_3'],
    ['intercept', 'Кривизна', 'Профиль пути', 'n_feature_3', 'n_feature_2'],
    ['intercept', 'Профиль пути', 'n_feature_3', 'n_feature_2']
]

target = ['Количество сшедших вагонов']

In [15]:
test_data = df[['intercept', 'Профиль пути', 'n_feature_3', 'n_feature_2', 'n_feature_1', 'Кривизна', 'Количество сшедших вагонов']].dropna()
len(test_data)

35

In [235]:
lambdas = [
    (lambda t: np.exp(t)),
    (lambda t: np.exp(-t * t)),
    (lambda t: 1 + np.sqrt(np.abs(25 - (t-5)*(t-5)))),
    (lambda t: (t-1)*(t-1)),
    (lambda t: 1 / (1 + t*t)),
    (lambda t: t * (np.pi / 2.0 - np.arctan(-t)) + 1),
    (lambda t: np.log(1 + t*t) + 1),
]

log_lambdas = [
    (lambda t: t),
    (lambda t: -t * t),
    (lambda t: np.log(1 + np.sqrt(np.abs(25 - (t-5)*(t-5))))),
    (lambda t: np.log((t-1)*(t-1))),
    (lambda t: -np.log(1 + t*t)),
    (lambda t: np.log(t * (np.pi / 2.0 - np.arctan(-t)) + 1)),
    (lambda t: np.log(np.log(1 + t*t) + 1)),
]

log_one_minus_lambdas = [
    (lambda t: np.log(np.abs(1 - np.exp(t)))),
    (lambda t: np.log(np.abs(1 - np.exp(-t * t)))),
    (lambda t: np.log(np.abs(-np.sqrt(np.abs(25 - (t-5)*(t-5)))))),
    (lambda t: np.log(np.abs(1 - (t-1)*(t-1)))),
    (lambda t: np.log(t*t) - np.log(1 + t*t)),
    (lambda t: np.log(np.abs(-t * (np.pi / 2.0 - np.arctan(-t))))),
    (lambda t: np.log(np.abs(-np.log(1 + t*t)))),
]

In [334]:
lambdas_2 = [
    (lambda t: np.exp(t)),
    (lambda t: np.exp(-t * t)),
    (lambda t: 1 / (1 + np.exp(-t))),
    #(lambda t: (t-1)*(t-1)),
    (lambda t: 1 / (1 + t*t)),
    (lambda t: t * (np.pi / 2.0 - np.arctan(-t)) + 1),
    #(lambda t: np.log(1 + t*t) + 1),
]

log_lambdas_2 = [
    (lambda t: t),
    (lambda t: -t * t),
    (lambda t: -np.log(1 + np.exp(-t))),
    #(lambda t: np.log((t-1)*(t-1))),
    (lambda t: -np.log(1 + t*t)),
    (lambda t: np.log(t * (np.pi / 2.0 - np.arctan(-t)) + 1)),
    #(lambda t: np.log(np.log(1 + t*t) + 1)),
]

log_one_minus_lambdas_2 = [
    (lambda t: np.log(1 - np.exp(t))),
    (lambda t: np.log(1 - np.exp(-t * t))),
    (lambda t: -np.log(1 + np.exp(t))),
    #(lambda t: np.log(1 - (t-1)*(t-1))),
    (lambda t: np.log(t*t) - np.log(1 + t*t)),
    (lambda t: np.log(-t * (np.pi / 2.0 - np.arctan(-t)))),
    #(lambda t: np.log(-np.log(1 + t*t))),
]

In [229]:
def toFixed2(a):
    result_str = ''
    #if type(a) == np.float64:
    #    return '[' + str(round(a, 2)) + ']'
    if a is None:
        return 'None'
    for i in a:
        result_str += ('%.2f, ' % i)
    result_str = result_str[:-2]
    result_str = '[' + result_str + ']'
    return result_str

In [305]:
class PoissonRegression():
    theta = None
    lambda_index = 0
    X = None
    y = None
    half_side = 1000
    bounds = 10 * [(-half_side, half_side)]
    new_bounds = 10 * [(None, None)]
    
    def __init__(self, df, features, target):
        self.df = df
        self.features = features
        self.target = target
        self.theta = np.zeros(len(self.features[0]))
    
    def log_likelihood_function_compact(self, theta):
        result = 0
        for i in range(len(self.y)):
            dot_prod = float(np.dot(self.X[i], theta))
            lambda_value = lambdas[self.lambda_index](dot_prod)
            log_lambda_value = log_lambdas[self.lambda_index](dot_prod)
            result += (float(self.y[i]) - 1) * log_lambda_value - lambda_value
        return -result
    
    def get_log_likelihood_const(self):
        result = 0
        for i in range(len(self.y)):
            result += -np.log(scipy.special.factorial(self.y[i]))
        return result
    
#     def construct_plot(self):
#         plt.yticks(range(1, 30,2))
#         plt.plot(data['Количество сшедших вагонов'], color='black', marker='^', linestyle='none', markersize=5)
#         #plt.plot(FIXME, color='red', marker='+', linestyle='none', markersize=5)
#         plt.xlabel('номер наблюдения')
#         plt.ylabel('кол-во сшедших вагонов')
#         plt.legend(['source' ,'lambda_1'], loc=5)
#         plt.show()
    
    def calc_aic_c(self, logL) -> float:
        logL = float(logL[1:-1])
        k = len(self.features)
        n = len(self.y)
        return round(2 * (k - logL) + 2 * k * (k + 1) / (n - k - 1), 2)
    
    def fit_all(self):
        for feature in self.features:
            data = self.df[feature + self.target].dropna()
            
            self.y = data[self.target]
            self.y = np.array(self.y)
            self.X = data.drop(self.target, axis=1)
            self.X = np.matrix(self.X)
            
            thetas_for_tex = ''
            thetas_without_round_for_tex = ''
            AIC_c_for_tex = ''
            
            print(feature)
            print('n = ', len(self.y))
            
            for lambda_index in range(len(lambdas)):
                self.lambda_index = lambda_index
                
                results = shgo(self.log_likelihood_function_compact, self.bounds[:len(feature)])
                thetas_for_tex += toFixed2(results['x']) + ' & '
                thetas_without_round_for_tex += str(results['x']) + ' & '
                AIC_c_for_tex += str(self.calc_aic_c(toFixed2(-results['fun'] + self.get_log_likelihood_const()))) + ' & '
                
                print('success: ', results['success'])
                print('ln L: ', toFixed2(-results['fun'] + self.get_log_likelihood_const()))
                print('AIC_c', self.calc_aic_c(toFixed2(-results['fun'] + self.get_log_likelihood_const())))
                print('thetas: ', toFixed2(results['x']))
                print()
                #min_llfc = minimize(self.log_likelihood_function_compact, np.zeros(len(feature)))
            
            print('thetas_for_tex: ', thetas_for_tex[:-2])
            print('AIC_c_for_tex:  ', AIC_c_for_tex[:-2])
            print('thetas_for_tex: ', thetas_without_round_for_tex[:-2])
            print("########################################################")

In [306]:
pRegressor = PoissonRegression(df, features, target)
pRegressor.fit_all()

['intercept', 'Кривизна']
n =  46
success:  True
ln L:  [-256.27]
AIC_c 532.43
thetas:  [1.18, -271.63]

success:  True
ln L:  [-306.66]
AIC_c 633.21
thetas:  [-0.00, 0.00]

success:  True
ln L:  [-306.66]
AIC_c 633.21
thetas:  [0.00, 0.00]

success:  True
ln L:  [-256.14]
AIC_c 532.17
thetas:  [-0.80, 208.15]

success:  True
ln L:  [-306.66]
AIC_c 633.21
thetas:  [-0.00, 0.00]

success:  True
ln L:  [-256.28]
AIC_c

<ipython-input-235-e1aeffde540f>:2: RuntimeWarning: overflow encountered in exp
  (lambda t: np.exp(t)),


 532.45
thetas:  [0.95, -236.29]

success:  True
ln L:  [-256.66]
AIC_c 533.21
thetas:  [2.66, -549.73]

thetas_for_tex:  [1.18, -271.63] & [-0.00, 0.00] & [0.00, 0.00] & [-0.80, 208.15] & [-0.00, 0.00] & [0.95, -236.29] & [2.66, -549.73] 
AIC_c_for_tex:   532.43 & 633.21 & 633.21 & 532.17 & 633.21 & 532.45 & 533.21 
thetas_for_tex:  [   1.17663977 -271.62551142] & [-2.73694797e-11  0.00000000e+00] & [0. 0.] & [ -0.79854568 208.15177928] & [-2.73694797e-11  0.00000000e+00] & [   0.94835424 -236.2893394 ] & [   2.66181393 -549.7289914 ] 
########################################################
['intercept', 'Кривизна', 'Профиль пути']
n =  41
success:  True
ln L:  [-242.56]
AIC_c 505.62
thetas:  [1.17, -160.85, 39.30]

success:  True
ln L:  [-299.87]
AIC_c 620.24
thetas:  [-0.00, 0.00, 0.00]



<ipython-input-235-e1aeffde540f>:2: RuntimeWarning: overflow encountered in exp
  (lambda t: np.exp(t)),


success:  True
ln L:  [-239.22]
AIC_c 498.94
thetas:  [0.56, -114.67, 35.95]

success:  True
ln L:  [-239.73]
AIC_c 499.96
thetas:  [-0.83, 212.70, -56.93]

success:  True
ln L:  [-299.87]
AIC_c 620.24
thetas:  [-0.00, 0.00, 0.00]

success:  True
ln L:  [-238.18]
AIC_c 496.86
thetas:  [1.05, -281.75, 83.44]

success:  True
ln L:  [-235.22]
AIC_c 490.94
thetas:  [3.28, -486.77, 276.36]

thetas_for_tex:  [1.17, -160.85, 39.30] & [-0.00, 0.00, 0.00] & [0.56, -114.67, 35.95] & [-0.83, 212.70, -56.93] & [-0.00, 0.00, 0.00] & [1.05, -281.75, 83.44] & [3.28, -486.77, 276.36] 
AIC_c_for_tex:   505.62 & 620.24 & 498.94 & 499.96 & 620.24 & 496.86 & 490.94 
thetas_for_tex:  [   1.17290277 -160.84988797   39.29626948] & [-2.73694797e-11  0.00000000e+00  0.00000000e+00] & [   0.55589183 -114.66800927   35.94991553] & [ -0.83371448 212.69978342 -56.92629995] & [-2.73694797e-11  0.00000000e+00  0.00000000e+00] & [   1.05191539 -281.7481183    83.4432927 ] & [   3.28364783 -486.769352    276.36319505]

...

success:  True
ln L:  [-213.12]
AIC_c 447.38
thetas:  [-0.15, -148.04, 53.13, 0.03]

success:  True
ln L:  [-283.07]
AIC_c 587.28
thetas:  [-0.00, 0.00, 0.00, -0.00]

success:  True
ln L:  [-247.44]
AIC_c 516.02
thetas:  [9.24, 3.94, 3.55, -0.11]

success:  True
ln L:  [-209.23]
AIC_c 439.6
thetas:  [0.29, 244.79, -52.47, -0.02]

success:  True
ln L:  [-283.07]
AIC_c 587.28
thetas:  [-0.00, 0.00, 0.00, -0.00]

success:  True
ln L:  [-208.26]
AIC_c 437.66
thetas:  [-0.47, -328.73, 75.88, 0.03]

success:  True
ln L:  [-213.65]
AIC_c 448.44
thetas:  [-0.31, -633.78, 7.19, 0.08]

thetas_for_tex:  [-0.15, -148.04, 53.13, 0.03] & [-0.00, 0.00, 0.00, -0.00] & [9.24, 3.94, 3.55, -0.11] & [0.29, 244.79, -52.47, -0.02] & [-0.00, 0.00, 0.00, -0.00] & [-0.47, -328.73, 75.88, 0.03] & [-0.31, -633.78, 7.19, 0.08] 
AIC_c_for_tex:   447.38 & 587.28 & 516.02 & 439.6 & 587.28 & 437.66 & 448.44 
thetas_for_tex:  [-1.48804956e-01 -1.48036460e+02  5.31272408e+01  2.66695834e-02] & [-7.09391652e-14  0.00000

<ipython-input-235-e1aeffde540f>:2: RuntimeWarning: overflow encountered in exp
  (lambda t: np.exp(t)),


success:  True
ln L:  [-281.07]
AIC_c 583.68
thetas:  [-0.00, 0.00, 0.00, -0.00, 0.00]

success:  True
ln L:  [-281.07]
AIC_c 583.68
thetas:  [0.00, 0.00, 0.00, 0.00, 0.00]

success:  True
ln L:  [-194.73]
AIC_c 411.0
thetas:  [-0.98, 242.26, -22.62, -0.01, 1.73]

success:  True
ln L:  [-281.07]
AIC_c 583.68
thetas:  [-0.00, 0.00, 0.00, -0.00, 0.00]

success:  True
ln L:  [-194.63]
AIC_c 410.8
thetas:  [1.28, -373.54, 41.10, 0.02, -2.34]

success:  True
ln L:  [-198.76]
AIC_c 419.06
thetas:  [2.69, -57.95, 235.44, 0.08, -8.15]

thetas_for_tex:  [0.92, -105.35, 34.34, 0.02, -2.24] & [-0.00, 0.00, 0.00, -0.00, 0.00] & [0.00, 0.00, 0.00, 0.00, 0.00] & [-0.98, 242.26, -22.62, -0.01, 1.73] & [-0.00, 0.00, 0.00, -0.00, 0.00] & [1.28, -373.54, 41.10, 0.02, -2.34] & [2.69, -57.95, 235.44, 0.08, -8.15] 
AIC_c_for_tex:   409.6 & 583.68 & 583.68 & 411.0 & 583.68 & 410.8 & 419.06 
thetas_for_tex:  [ 9.20623915e-01 -1.05351925e+02  3.43369654e+01  2.12149958e-02
 -2.23822805e+00] & [-6.99064063e-14

<ipython-input-235-e1aeffde540f>:2: RuntimeWarning: overflow encountered in exp
  (lambda t: np.exp(t)),


success:  True
ln L:  [-205.16]
AIC_c 431.46
thetas:  [-0.49, -15.96, -0.02, 1.44]

success:  True
ln L:  [-283.07]
AIC_c 587.28
thetas:  [-0.00, 0.00, -0.00, 0.00]

success:  True
ln L:  [-205.19]
AIC_c 431.52
thetas:  [0.31, 26.23, 0.03, -1.97]

success:  True
ln L:  [-204.94]
AIC_c 431.02
thetas:  [1.91, 281.49, 0.08, -6.90]

thetas_for_tex:  [0.55, 39.28, 0.03, -2.14] & [-0.00, 0.00, -0.00, 0.00] & [0.00, 0.00, 0.00, 0.00] & [-0.49, -15.96, -0.02, 1.44] & [-0.00, 0.00, -0.00, 0.00] & [0.31, 26.23, 0.03, -1.97] & [1.91, 281.49, 0.08, -6.90] 
AIC_c_for_tex:   422.54 & 587.28 & 587.28 & 431.46 & 587.28 & 431.52 & 431.02 
thetas_for_tex:  [ 5.51509058e-01  3.92803528e+01  2.54177579e-02 -2.13831493e+00] & [-7.07737609e-14  0.00000000e+00 -2.01492897e-10  0.00000000e+00] & [0. 0. 0. 0.] & [ -0.49176887 -15.96251758  -0.01698865   1.43914189] & [-7.04401905e-14  0.00000000e+00 -2.00519742e-10  0.00000000e+00] & [ 0.31226161 26.22997155  0.02698085 -1.96985398] & [ 1.90837587e+00  2.81493

In [337]:
class GeometryRegression():
    theta = None
    lambda_index = 0
    X = None
    y = None
    half_side = 1000
    bounds = 10 * [(-half_side, half_side)]
    new_bounds = 10 * [(None, None)]
    
    def __init__(self, df, features, target):
        self.df = df
        self.features = features
        self.target = target
        self.theta = np.zeros(len(self.features[0]))
    
    def log_likelihood_function(self, theta) -> float:
        result = 0
        for i in range(len(self.y)):
            dot_prod = float(np.dot(self.X[i], theta))
            lambda_value = lambdas_2[self.lambda_index](dot_prod)
            log_lambda_value = log_lambdas_2[self.lambda_index](dot_prod)
            #if 1 - np.exp(dot_prod) <= 1e-30:
            #    print('ALARM !!!!!!!!!!!!!!!!!!')
            #result += (float(self.y[i]) - 1) * np.log(1 - np.exp(dot_prod)) + dot_prod
            #result += (float(self.y[i]) - 1) * np.log(1 - np.exp(dot_prod) / (1 + np.exp(dot_prod))) + dot_prod - np.log(1 + np.exp(dot_prod))
            result += (float(self.y[i]) - 1) * log_one_minus_lambdas_2[self.lambda_index](dot_prod) + log_lambda_value
        return -result
    
#     def construct_plot(self):
#         plt.yticks(range(1, 30,2))
#         plt.plot(data['Количество сшедших вагонов'], color='black', marker='^', linestyle='none', markersize=5)
#         #plt.plot(FIXME, color='red', marker='+', linestyle='none', markersize=5)
#         plt.xlabel('номер наблюдения')
#         plt.ylabel('кол-во сшедших вагонов')
#         plt.legend(['source' ,'lambda_1'], loc=5)
#         plt.show()
    
    def calc_aic_c(self, logL) -> float:
        logL = float(logL[1:-1])
        k = len(self.features)
        n = len(self.y)
        return round(2 * (k - logL) + 2 * k * (k + 1) / (n - k - 1), 2)
    
    def fit_all(self):
        for feature in self.features:
            data = self.df[feature + self.target].dropna()
            
            self.y = data[self.target]
            self.y = np.array(self.y)
            self.X = data.drop(self.target, axis=1)
            self.X = np.matrix(self.X)
            
            thetas_for_tex = ''
            thetas_without_round_for_tex = ''
            AIC_c_for_tex = ''
            
            print(feature)
            print('n = ', len(self.y))
            
            for lambda_index in range(len(lambdas_2)):
                self.lambda_index = lambda_index
                # shgo, dual_annealing, differential_evolution, basinhopping
                results = dual_annealing(self.log_likelihood_function, self.bounds[:len(feature)])
                i = 0
                while math.isnan(results['fun']) and i < 10:
                    results = dual_annealing(self.log_likelihood_function, self.bounds[:len(feature)])
                    print('lambda_index = ', lambda_index)
                    i += 1
                thetas_for_tex += toFixed2(results['x']) + ' & '
                thetas_without_round_for_tex += str(results['x']) + ' & '
                AIC_c_for_tex += str(self.calc_aic_c(toFixed2([-results['fun']]))) + ' & '
                
                print('success: ', results['success'])
                print('ln L: ', toFixed2([-results['fun']]))
                print('AIC_c: ', self.calc_aic_c(toFixed2([-results['fun']])))
                print('thetas: ', toFixed2(results['x']))
                print()
                #min_llfc = minimize(self.log_likelihood_function, np.zeros(len(feature)))
            
            print('thetas_for_tex: ', thetas_for_tex[:-2])
            print('AIC_c_for_tex:  ', AIC_c_for_tex[:-2])
            print('thetas_for_tex: ', thetas_without_round_for_tex[:-2])
            print("########################################################")

In [338]:
gRegressor = GeometryRegression(df, features, target)
gRegressor.fit_all()

['intercept', 'Кривизна']
n =  46


<ipython-input-334-49b39c550010>:2: RuntimeWarning: overflow encountered in exp
  (lambda t: np.exp(t)),
<ipython-input-334-49b39c550010>:22: RuntimeWarning: overflow encountered in exp
  (lambda t: np.log(1 - np.exp(t))),
<ipython-input-334-49b39c550010>:22: RuntimeWarning: invalid value encountered in log
  (lambda t: np.log(1 - np.exp(t))),


lambda_index =  0
lambda_index =  0
success:  True
ln L:  [-98.07]
AIC_c:  216.03
thetas:  [-1.46, 215.57]

success:  True
ln L:  [-98.19]
AIC_c:  216.27
thetas:  [1.20, -90.37]



<ipython-input-334-49b39c550010>:24: RuntimeWarning: overflow encountered in exp
  (lambda t: -np.log(1 + np.exp(t))),
<ipython-input-337-fedfc962834c>:26: RuntimeWarning: invalid value encountered in double_scalars
  result += (float(self.y[i]) - 1) * log_one_minus_lambdas_2[self.lambda_index](dot_prod) + log_lambda_value
<ipython-input-334-49b39c550010>:4: RuntimeWarning: overflow encountered in exp
  (lambda t: 1 / (1 + np.exp(-t))),
<ipython-input-334-49b39c550010>:14: RuntimeWarning: overflow encountered in exp
  (lambda t: -np.log(1 + np.exp(-t))),


success:  True
ln L:  [-98.21]
AIC_c:  216.31
thetas:  [-1.18, 278.32]

success:  True
ln L:  [-98.04]
AIC_c:  215.97
thetas:  [-1.82, 233.20]



...

lambda_index =  0
success:  True
ln L:  [-282.89]
AIC_c:  586.92
thetas:  [3.97, -388.56, -0.13, -13.95]

success:  True
ln L:  [-84.25]
AIC_c:  189.64
thetas:  [0.18, 49.54, 0.03, -1.64]



<ipython-input-334-49b39c550010>:24: RuntimeWarning: overflow encountered in exp
  (lambda t: -np.log(1 + np.exp(t))),
<ipython-input-337-fedfc962834c>:26: RuntimeWarning: invalid value encountered in double_scalars
  result += (float(self.y[i]) - 1) * log_one_minus_lambdas_2[self.lambda_index](dot_prod) + log_lambda_value
<ipython-input-334-49b39c550010>:4: RuntimeWarning: overflow encountered in exp
  (lambda t: 1 / (1 + np.exp(-t))),
<ipython-input-334-49b39c550010>:14: RuntimeWarning: overflow encountered in exp
  (lambda t: -np.log(1 + np.exp(-t))),
C:\ProgramData\Anaconda3\lib\site-packages\scipy\optimize\optimize.py:697: RuntimeWarning: invalid value encountered in double_scalars
  df = (f(*((xk + d,) + args)) - f0) / d[k]


lambda_index =  2
lambda_index =  2
success:  True
ln L:  [-77.49]
AIC_c:  176.12
thetas:  [-0.59, -15.12, -0.02, 1.88]

success:  True
ln L:  [-76.32]
AIC_c:  173.78
thetas:  [-1.73, 36.23, 0.08, -1.76]



<ipython-input-334-49b39c550010>:27: RuntimeWarning: invalid value encountered in log
  (lambda t: np.log(-t * (np.pi / 2.0 - np.arctan(-t)))),


success:  True
ln L:  [-78.25]
AIC_c:  177.64
thetas:  [-0.78, 2.12, -0.01, 0.85]

thetas_for_tex:  [3.97, -388.56, -0.13, -13.95] & [0.18, 49.54, 0.03, -1.64] & [-0.59, -15.12, -0.02, 1.88] & [-1.73, 36.23, 0.08, -1.76] & [-0.78, 2.12, -0.01, 0.85] 
AIC_c_for_tex:   586.92 & 189.64 & 176.12 & 173.78 & 177.64 
thetas_for_tex:  [ 3.96909968e+00 -3.88558572e+02 -1.25722501e-01 -1.39467001e+01] & [ 1.81064645e-01  4.95397475e+01  2.73994622e-02 -1.63731255e+00] & [ -0.58782113 -15.12305804  -0.02304889   1.88216471] & [-1.73499905 36.22822185  0.08070946 -1.75845933] & [-0.78365416  2.12039468 -0.00883254  0.8487121 ] 
########################################################
